In [1]:
from IPython.display import display
import csv
import numpy as np
import ipyleaflet as leaflet
import ipywidgets as widgets

# Static variables for parsing
state_list = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
desired_keys = ['Severity', 'Start_Lat', 'End_Lat', 'Start_Lng', 'End_Lng', 'City', 'State']
data =[]

# Parse data as csv file
with open('US_Accidents_May19.csv', mode='r') as file:
    csv_reader = csv.reader(file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        temp = {}
        if line_count==0:
            keys = row
        else:
            for i in range(len(keys)):
                temp[keys[i]] = row[i]
            data_sample = {key: temp[key] for key in desired_keys}
            del temp
            data.append(data_sample)
        line_count += 1
        
# Initialize map and layer object
m = leaflet.Map(center=(0,0), zoom=1)
m.add_control(leaflet.LayersControl())
layer_group = leaflet.LayerGroup()

# Filter dictionary
filter_value = {'State': 'FL', 'City': 'Jacksonville', 'Severity': 4}
marker_color = {1: 'green', 2: 'blue', 3: 'yellow', 4: 'red'}

# Filter function
def predicate(sample):
    if sample['State']==filter_value['State'] and sample['City']==filter_value['City'] and int(sample['Severity'])==filter_value['Severity']:
        return True
    else:
        return False

# Filtering
def filter_input(data): return list(filter(predicate, data))

# Update Map
def update_map(state, city, severity):
    # Update filter value
    filter_value['State'], filter_value['City'], filter_value['Severity'] = state, city, severity
    color = marker_color[severity]
    
    # Filter data by input param
    output = filter_input(data)
    if len(output)==0:
        print('Warning: no accident, which can be caused by wrong city name')
        return
    print('Numer of accidents: {}'.format(len(output)))
    
    # Get center and zoom
    # Get coordinates from filter output (lat, long)
    coordinates = []
    lat_center, lng_center = 0, 0
    min_lat, max_lat, min_lng, max_lng = 0, 0, 0, 0
    for sample in output:
        coordinates.append((float(sample['Start_Lat']), float(sample['Start_Lng'])))
        lat_center += float(sample['Start_Lat'])
        lng_center += float(sample['Start_Lng'])
    lat_center /= len(output)
    lng_center /= len(output)

    # Get min and max lat for zoom calculation
    lat_min_max = min(coordinates)[0], max(coordinates)[0]
    lat_range = lat_min_max[1]-lat_min_max[0]
    lng_min_max = min(coordinates)[1], max(coordinates)[1]
    lng_range = lng_min_max[1]-lng_min_max[0]
    if lat_range > lng_range:
        max_range = lat_range
    else:
        max_range = lng_range

    # Generate center and zoom for map
    center = (lat_center, lng_center) # geological center of US
    zoom = int(np.floor(-3.3333*max_range + 11.6667))
    if zoom > 18:
        zoom = 18
    elif zoom < 1:
        zoom = 1
        
    # Reset map
    if len(m.layers) > 1:
        m.remove_layer(m.layers[1])
    m.center = center
    m.zoom = zoom

    # Clean the circles layer group from previous trigger
    if len(layer_group.layers) != 0:
        for layer in layer_group.layers:
            layer_group.remove_layer(layer)

    # Add new circles to layer group
    for i in range(len(coordinates)):
        layer_group.add_layer(leaflet.Circle(location=coordinates[i], radius=30, color=color, fill_color=color))

    # Add layer group to map
    m.add_layer(layer_group)
    
# Create widgets for map
state_widget = widgets.Dropdown(
    options=state_list,
    value='AL',
    description='State:',
    disabled=False,
)

city_widget = widgets.Text(
    #value='',
    placeholder='e.g. Jacksonville',
    description='City:',
    disabled=False
)

severity_widget = widgets.BoundedIntText(
    value=1,
    min=1,
    max=4,
    step=1,
    description='Severity',
    disabled=False
)

button_widget = widgets.Button(description="Update")
def on_button_clicked(b):
    update_map(state_widget.value, city_widget.value, severity_widget.value)
button_widget.on_click(on_button_clicked)

# Create widget control
state_widget_control = leaflet.WidgetControl(widget=state_widget, position='topright')
city_widget_control = leaflet.WidgetControl(widget=city_widget, position='topright')
severity_widget_control = leaflet.WidgetControl(widget=severity_widget, position='topright')
button_widget_control = leaflet.WidgetControl(widget=button_widget, position='topright')

# Add widget control
m.add_control(state_widget_control)
m.add_control(city_widget_control)
m.add_control(severity_widget_control)
m.add_control(button_widget_control)

# Display map
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …